In [1]:
import cv2, time, os, tensorflow as tf
import numpy as np
from tensorflow.python.keras.utils.data_utils import get_file

In [2]:
np.random.seed(30)

In [32]:
# classesFilePath = './coco.names'
classesFilePath = './my_class.names'

# modelURL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz"
# modelURL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz"
modelURL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz"
imagePath = "../../Data/helmet/images/BikesHelmets10.png"

In [33]:
with open(classesFilePath, "r") as f:
    classesList = f.read().splitlines()

colorList = np.random.uniform(low=0, high=255, size=(len(classesList), 3))
print(len(classesList), len(colorList))

3 3


### Downloading TFOD Model

In [37]:
fileName = os.path.basename(modelURL)
# modelName = fileName[:fileName.index('.')]
modelName = "my_model1"
print("fileName: ", fileName, "ModelName: ", modelName)

cacheDir = "./pretrained_models"
# Creating the models directory if not existed before
os.makedirs(cacheDir, exist_ok=True)
# get_file(fileName, origin=modelURL, cache_dir=cacheDir, cache_subdir="models", extract=True)

fileName:  ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz ModelName:  my_model1


In [38]:
tf.keras.backend.clear_session()
model = tf.saved_model.load(os.path.join(cacheDir, "models", modelName, "saved_model"))

In [7]:
tf.keras.backend.clear_session()
model1 = tf.saved_model.load(os.path.join(cacheDir, "models", modelName, "saved_model"))

### Detecting from Image

In [39]:
# Loading and converting an image to tensor for test purpose
image = cv2.imread(imagePath)
imageRGB = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2RGB)
imageTensor = tf.convert_to_tensor(imageRGB, dtype=tf.uint8)
imageTensor = imageTensor[tf.newaxis, ...]

In [40]:
detections = model(imageTensor)

In [41]:
def createBoundingBox(image, detections, threshold1 = 0.5, threshold2=0.4):
    bboxes = detections['detection_boxes'][0].numpy()
    classIndexes = detections['detection_classes'][0].numpy().astype(np.int32)
    classScores = detections['detection_scores'][0].numpy()

    imH, imW, imC = image.shape
    image = cv2.resize(image, (1000, int(imH * (1000/imW))))
    imH, imW = (int(imH * (1000/imW)), 1000)

    # gives indexes of bboxes with the criteria
    bboxIds = tf.image.non_max_suppression(bboxes, classScores, max_output_size=50, iou_threshold=threshold1, score_threshold=threshold2)


    if len(bboxes):
        for i in bboxIds:
            bbox = tuple(bboxes[i])
            classIndex = classIndexes[i]
            classLabel = classesList[classIndex]
            classConfidence = round(100*classScores[i])
            classColor = colorList[classIndex]

            displayTxt = f'{classLabel}: {classConfidence}%'
            # displayTxt = '{}: {}%'.format(classLabel, classConfidence)
            ymin, xmin, ymax, xmax = bbox   # in relative format
            ymin, xmin, ymax, xmax = (int(ymin*imH), int(xmin*imW), int(ymax*imH), int(xmax*imW))

            # if classLabel in ["motorcycle", "bicycle", "person"]:
            if classLabel in classesList:
                cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color=classColor, thickness=1)
                cv2.putText(image, displayTxt, (xmin, ymin - 7), cv2.FONT_HERSHEY_COMPLEX,  1, classColor, 2)
            # cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color=classColor, thickness=1)
            # cv2.putText(image, displayTxt, (xmin, ymin - 7), cv2.FONT_HERSHEY_COMPLEX,  1, classColor, 2)

    cv2.imwrite("./detections/" + modelName + ".jpg", image)
    cv2.imshow("Object detection", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [18]:
print(classesList)

['__background__', 'With Helmet', 'Without Helmet']


In [42]:
createBoundingBox(image, detections)

### Detecting in video

In [43]:
vdoPath = "../../Data/detect_04.mp4"

### Collecting Frames

In [40]:
def collectFrames(vdoPath, interval=1):
    vdo = cv2.VideoCapture(vdoPath)
    fps = vdo.get(cv2.CAP_PROP_FPS)
    fInterval = int(interval*fps)

    frames = []
    fno = 0
    while True:
        captured, frame = vdo.read()
        if not captured:
            break

        if fno%fInterval == 0:
            frames.append(frame)
        fno += 1

    vdo.release()
    cv2.destroyAllWindows()

    return frames

In [41]:
frames = collectFrames(vdoPath)

In [42]:
print(len(frames))
print(frames[0].shape)

10
(720, 1280, 3)


In [44]:
vdo = cv2.VideoCapture(vdoPath)

t1 = 0
while True:
    captured, frame = vdo.read()

    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    imageRGB = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2RGB)
    imageTensor = tf.convert_to_tensor(imageRGB, dtype=tf.uint8)
    imageTensor = imageTensor[tf.newaxis, ...]

    detections = model(imageTensor)

    bboxes = detections['detection_boxes'][0].numpy()
    # print("len: ", bboxes)
    classIndexes = detections['detection_classes'][0].numpy().astype(np.int32)
    classScores = detections['detection_scores'][0].numpy()
    print("index", classIndexes)

    imH, imW, imC = image.shape
    # image = cv2.resize(image, (1000, int(imH * (1000/imW))))
    # imH, imW = (int(imH * (1000/imW)), 1000)

    # gives indexes of bboxes with the criteria
    bboxIds = tf.image.non_max_suppression(bboxes, classScores, max_output_size=50, iou_threshold=0.5, score_threshold=0.5)
    print(len(bboxIds))


    if len(bboxes):
        for i in bboxIds:
            bbox = tuple(bboxes[i])
            classIndex = classIndexes[i]
            classLabel = classesList[classIndex]
            classConfidence = round(100*classScores[i])
            classColor = colorList[classIndex]

            displayTxt = f'{classLabel}: {classConfidence}%'
            # displayTxt = '{}: {}%'.format(classLabel, classConfidence)
            ymin, xmin, ymax, xmax = bbox   # in relative format
            ymin, xmin, ymax, xmax = (int(ymin*imH), int(xmin*imW), int(ymax*imH), int(xmax*imW))
            print(f"box: {ymin}, {xmin}, {ymax}, {xmax}, index: {classIndex}, label: {classLabel}")

            target = ["motorcycle", "bicycle", "person"]
            if classLabel in classesList:
            # if classLabel in target:
                cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), color=classColor, thickness=1)
                cv2.putText(frame, displayTxt, (xmin, ymin - 7), cv2.FONT_HERSHEY_COMPLEX_SMALL,  1, classColor, 1)
            # cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), color=classColor, thickness=1)
            # cv2.putText(frame, displayTxt, (xmin, ymin - 7), cv2.FONT_HERSHEY_COMPLEX_SMALL,  1, classColor, 1)


    t2 = time.time()
    fps = 1/(t2 - t1)
    t1 = t2

    cv2.putText(frame, f"FPS: {int(fps)}", (30, 30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 200, 0), 1)
    cv2.imshow("Video", frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cv2.waitKey(0)
vdo.release()
cv2.destroyAllWindows()

index [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
0
index [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
0
index [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
0
index [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
0
index [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 